# SMdM analysis of data, produced by Concatenate_and_find_peaks.ipynb script

This program takes an .hdf5 file with xy coordinates of fluorescent peaks as an input and preform these steps:

- Cluster point using Voronoi clusterng procedure
- Align defined clusters along x-axis
- Calculate displacements of fluorescent peaks between odd and even frames in accordance with SMdM
- Fit displacement data in each cluster with normalized probability density function (more details in the code)
- Make displacement maps for each cluster with desired bin size 
- Analyze selected regions in each cluster and perform displacement fitting in them

Rijksuniversiteit Groningen, 2021

C.M. Punter (c.m.punter@rug.nl)

D.S. Linnik (d.s.linnik@rug.nl)

W.M. Smigiel (w.m.smigiel@gmail.com)

L. Mantovanelli (l.mantovanelli@rug.nl)

In [ ]:
import os
import h5py
import shutil
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, voronoi_plot_2d

########################################################################################
########################################################################################
##################### SETTING IMPORTANT PARAMETERS OF THE ANALYSIS #####################

pixel_size = 100e-3 # in micrometers
delta_t = 1.5 # in milliseconds
delta_t /= 1000

### Selecting the hdf5 file ###
root_path = os.getcwd() # Determine path to the .hdf5 files

# Find a hdf5 file in directory or initialize in mannualy 
path_hdf5 = "No_file"

for out in os.listdir(root_path):
    if out.endswith('.hdf5'):
        path_hdf5 = r'{}'.format(out)       
# path_hdf5 = r"20201023_3.hdf5" # Mannualy determine .hdf5 file
if path_hdf5 == "No_file":
    raise Exception('No hdf5 file is found in current directory')

### Parameters that are used for the voronoi clustering of cells ###

cluster_density_factor = 0.15
cluster_min_size = 100

### Parameters that are used for pairing peaks ###

r_max = 0.6 # displasements restrictions in micrometers
background_correction = True

filter_pairs= not background_correction # when background correction is not used, ambiguous pairs are filtered

# initial parameters that used for fitting of the data
initial_D = 1            # initial diffusion coefficients used for fitting      
initial_b = 0            # initial background correction for fitting
    
# If all the clustered point clouds should be analysed
analyse_all_cells = True

# parameters for making diffusion maps
pixel_bin_size_nm = 200      # in nm
pixel_bin_size = pixel_bin_size_nm*100e-5 # in micrometers
outlier_correction = True # For Tukey's fences outlier detection 
k_value = 3  # k value for Tukey's fences outlier detection  

# parameters for poles selection
Area_selection = True    # do poles need to be analysed
persantage_of_pole = 0.2 # persantage of a pole in a cell - default value is 0.2


################## END OF SETTING IMPORTANT PARAMETERS OF THE ANALYSIS #################
########################################################################################
########################################################################################

# Create a tree of directorys for further storage of outputs

# Remove already made folders inside "Plots" folder (Exsept plots for points densety mappng,clustering and aligned cells)
# Some parts can be commented in order not to delete info while reanalyzing data 
try:
    shutil.rmtree(root_path+"/"+path_hdf5[:-5]+'/Plots/Global fitting of cells')
except FileNotFoundError:
    pass
try:
    shutil.rmtree(root_path+"/"+path_hdf5[:-5]+'/Plots/Fitting of areas')
except FileNotFoundError:
    pass
try:
    shutil.rmtree(root_path+"/"+path_hdf5[:-5]+'/Plots/Diffusion maps')
except FileNotFoundError:
    pass

# Remove made folder for CSV files
# Can be commented in order not to delete info while reanalyzing data 
try:
    shutil.rmtree(root_path+"/"+path_hdf5[:-5]+'/CSV files')
except FileNotFoundError:
    pass

# Creating new folders to save analysis output
os.chdir(root_path)
try:
    os.mkdir(path_hdf5[:-5])
except FileExistsError:
    pass
os.chdir(path_hdf5[:-5])
try:
    os.mkdir("Intermediate files")
except FileExistsError:
    pass
try:
    os.mkdir("CSV files")
except FileExistsError:
    pass
try:
    os.mkdir("Plots")
except FileExistsError:
    pass
os.chdir("Plots")
try:
    os.mkdir("Global fitting of cells")
except FileExistsError:
    pass
try:
    os.mkdir("Diffusion maps")
except FileExistsError:
    pass
os.chdir(root_path)

## Open the HDF5 file with all the fitted peaks and determine the total number of frames

In [ ]:

f = h5py.File(path_hdf5, 'r')

# get all frame numbers from the hdf5 file
frames = [int(key[3:]) for key in f.keys() if key.startswith("fr_")]
n_frames = max(frames)
print("Total number of frames is %a" % (n_frames))


# get all x, y coordinates from the HDF5 file
x = []
y = []
frame = []

for i in range(0, n_frames): 
    if "fr_%d" % i in f:
        x1 = np.array(f["fr_%d" % i]['x'])
        y1 = np.array(f["fr_%d" % i]['y'])
    else:
        x1 = np.array(())
        y1 = np.array(())
    x.extend(x1)
    y.extend(y1)
    frame.extend([i] * x1.size)

frame = np.array(frame)
x = np.array(x)
y = np.array(y)


print("Number of frames in analysis is %a" % (n_frames))

# plot 2D histogram to visualize the data
fig, ax = plt.subplots()
fig.set_figwidth(15)
fig.set_figheight(10)
h = ax.hist2d(x, y, (round(x.max() - x.min()), round(y.max() - y.min())), cmap="Greys")
# plt.colorbar(h[3], ax=ax) # optional adding of a colorbar on the left
ax.set(xlabel='x', ylabel='y', title='Localizations')

# savinf of a plot to folder "Plots"
name_of_dir_to_save = path_hdf5[:-5]+"/Plots/"
plt.savefig(name_of_dir_to_save+path_hdf5[:-5]+', Localization of cells.pdf')

plt.show() # show plot in a notebook


## We use voronoi clustering to detect cells. The voronoi clustering code is based on 
## https://github.com/ZhuangLab/storm-analysis/blob/master/storm_analysis/voronoi/voronoi_analysis.py .

In [ ]:

# based on https://github.com/ZhuangLab/storm-analysis/blob/master/storm_analysis/voronoi/voronoi_analysis.py
def voronoi_clustering(x, y, density_factor, min_size):

    points = np.stack((x, y), axis=-1)
    vor = Voronoi(points) 
    
    # https://en.wikipedia.org/wiki/Shoelace_formula
    def polygon_area(x, y):
        return 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))

    # determine density of each region
    density = np.zeros(x.size)
    labels = np.zeros(x.size, dtype = np.int32) - 1
    
    for i, region_index in enumerate(vor.point_region):
        vertices = []

        for vertex in vor.regions[region_index]:
            if (vertex != -1):
                vertices.append(vor.vertices[vertex])

        vertices = np.array(vertices)
        area = polygon_area(vertices[:,0], vertices[:,1])
        density[i] = 1 / area

    # determine minimum density threshold
    median_density = np.median(density)
    threshold_density = median_density * density_factor
    
    max_neighbors = 40
    neighbors = np.zeros((x.size, max_neighbors), dtype = np.int32) - 1
    neighbors_counts = np.zeros((x.size), dtype = np.int32)

    for ridge_p in vor.ridge_points:
        p1 = ridge_p[0]
        p2 = ridge_p[1]

        # add p2 to the list for p1
        neighbors[p1,neighbors_counts[p1]] = p2
        neighbors_counts[p1] += 1

        # add p1 to the list for p2
        neighbors[p2,neighbors_counts[p2]] = p1
        neighbors_counts[p2] += 1

    min_density = density_factor * median_density
    visited = np.zeros(x.size, dtype = np.int32)

    def neighborsList(index):
        nlist = []
        for i in range(neighbors_counts[index]):
            loc_index = neighbors[index,i]
            if (visited[loc_index] == 0):
                nlist.append(neighbors[index,i])
                visited[loc_index] = 1
        return nlist

    cluster_id = 0
    for i in range(x.size):
        if (visited[i] == 0):
            visited[i] = 1
            if (density[i] > min_density):
                cluster_elt = [i]
                c_size = 1
                to_check = neighborsList(i)
                while (len(to_check) > 0):

                    # remove last localization from the list.
                    loc_index = to_check[-1]
                    to_check = to_check[:-1]

                    # if the localization has sufficient density add to cluster and
                    # check neighbors.
                    if (density[loc_index] > min_density):
                        to_check += neighborsList(loc_index)
                        cluster_elt.append(loc_index)
                        c_size += 1

                    # mark as visited.
                    visited[loc_index] = 1

                # mark the cluster if there are enough localizations in the cluster.
                if (c_size > min_size):
                    for elt in cluster_elt:
                        labels[elt] = cluster_id
                    cluster_id += 1
    
    return labels


cell = voronoi_clustering(x, y, cluster_density_factor, cluster_min_size)

# plot all found clusters
fig, ax = plt.subplots()
fig.set_figwidth(15)
fig.set_figheight(10)
ax.hist2d(x, y, (round(x.max() - x.min()), round(y.max() - y.min())), cmap='Greys')
ax.scatter(x[cell >= 0], y[cell >= 0], c=cell[cell >= 0], s=5, cmap="Paired")
ax.set(xlabel='x', ylabel='y', title='Clustered Cells')

# show cluster number on a plot
for c in np.unique(cell):
    if c >= 0:
        mask = cell == c
        x1 = x[mask].mean()
        y1 = y[mask].mean()
        ax.text(x1, y1, f"$\\bf{c}$", color="red", size = 15)

# save plot in folder "Plots"
name_of_dir_to_save = path_hdf5[:-5]+"/Plots/"
plt.savefig(name_of_dir_to_save+path_hdf5[:-5]+', Voronoi clustering of cells.png')

plt.show()

# filter peaks that are not within a cluster
frame = frame[cell >= 0]
x = x[cell >= 0]
y = y[cell >= 0]
cell = cell[cell >= 0]



## Align all cells along the x-axis.

In [ ]:
def rotate(theta, points):
    """
        Rotates the array of (x,y) points at angle theta (in radians) using rotation matrix
    """ 
    r = np.array(((np.cos(theta), -np.sin(theta)),
                 (np.sin(theta),  np.cos(theta))))
    return points.dot(r)

# https://alyssaq.github.io/2015/computing-the-axes-or-orientation-of-a-blob/
def find_orientation(points):
    """
        Gets the angle of first principle component of a 2D pointcloud as an 
        angle of the first eigenvector of a covariance matrix of xy points  
    """
    cov = np.cov(points[:,0], points[:,1])
    evals, evecs = np.linalg.eig(cov)
    sort_indices = np.argsort(evals)
    x, y = evecs[:, sort_indices[1]]    
    return np.arctan2(y, x)


# create a grid to plot each rotated cluster seperately
number_of_cells = np.unique(cell).size
cols = 4
rows = ((number_of_cells - 1) // cols) + 1
fig, axes = plt.subplots(nrows=rows, ncols=cols, sharex=True, sharey=True, figsize = (9, 9))
fig.set_figwidth(15)
fig.set_figheight(10)

# initialize dictionaries to save intermediate data 
Dict_for_number_of_points = {}
Dict_for_rotation_angles ={}

# plotting rotated clusters
for i, c in enumerate(np.unique(cell)):
    cell_mask = cell == c
    points = np.stack((x[cell_mask], y[cell_mask]), axis=1)
    points -= np.mean(points, axis=0)
    points *= pixel_size
    
    theta = find_orientation(points)        
    points = rotate(theta, points)
    x[cell_mask], y[cell_mask] = points.T
    
    Dict_for_number_of_points[c] = len(x[cell_mask])
    Dict_for_rotation_angles[c] = round(theta*180/np.pi,2)
    
    ax = axes[i // cols, i % cols]
    ax.scatter(x[cell_mask], y[cell_mask],s=0.1)
    ax.set_aspect("equal")
    if theta >=0:
        ax.set_title('Cell %d, ' % c + '{0:.0f} points, '.format(len(x[cell_mask])) + str(abs(Dict_for_rotation_angles[c])) +  chr(176) +'\u2193')
    else:
        ax.set_title('Cell %d, ' % c + '{0:.0f} points, '.format(len(x[cell_mask])) + str(abs(Dict_for_rotation_angles[c])) +  chr(176) +'\u2191')
        

for row in range(rows):
    axes[row, 0].set_ylabel("\u03BCm")
for col in range(cols):
    axes[rows - 1, col].set_xlabel("\u03BCm")

fig.suptitle("Aligned cells (" + "\u2193 for clockwise rotation, \u2191 for counter clockwise rotation )")
plt.tight_layout()

# save plot in folder "Plots"
name_of_dir_to_save = path_hdf5[:-5]+"/Plots/"
plt.savefig(name_of_dir_to_save+path_hdf5[:-5]+', Table of alignes cells.png')

plt.show()


## Calculating displacements either with point discarding or background correction (based on setted parematers)
## and saving intermediate files for reanalyzing ease

In [ ]:
# asking user wether all clusters should be abalysed unless analyse_all_cells is TRUE
if analyse_all_cells:
    list_of_cells = list(np.unique(cell))
else: 
    str_list_of_cells = input("Enter perspective cells numbers with space or all\n")  
    if str_list_of_cells == "all":
        list_of_cells = list(np.unique(cell))
    else:
        list_of_cells = [int(x) for x in str_list_of_cells.split()]

counter_cells = 0

print("\n")

displacements = []
if filter_pairs:
    print("Discarding pairs outside radius of %a nm" % (r_max*1000))
    
for c in np.unique(cell):
    if c in list_of_cells:
        
        # setting the progress bar
        percantage_to_text = int(counter_cells*100/len(list_of_cells))
        percantage_to_plot = int(counter_cells*100/len(list_of_cells)/2)
        print("Cell %a, " %c +"\u25ae" * percantage_to_plot + "\u25af" * (50-percantage_to_plot) + " " + str(percantage_to_text) + "% \r", end="")
        counter_cells += 1
        
        for fr in range(0, n_frames,2):
                      
            first = (frame == fr) & (cell == c)
            second = (frame == (fr + 1)) & (cell == c)

            x1, y1, x2, y2 = x[first], y[first], x[second], y[second]

            # calculate a distance matrix
            x_from, x_to = np.meshgrid(x1, x2)
            y_from, y_to = np.meshgrid(y1, y2)
            r = np.sqrt(((x_to - x_from) ** 2) + ((y_to - y_from) ** 2))

            # check which coordinates are within r_max distance
            connected = r <= r_max
            valid = connected

            if filter_pairs:
                # make sure each coordinate is linked to exactly one other coordinate
                single_from = np.sum(connected, axis=0) == 1
                single_to = np.sum(connected, axis=1) == 1
                valid = connected & np.logical_and(*np.meshgrid(single_from, single_to))

            pairs = np.array((np.full(valid.sum(), fr), np.full(valid.sum(), c), x_from[valid], y_from[valid], x_to[valid], y_to[valid], r[valid])).T
            displacements.extend(pairs)
            

print("Cell %a, " % list_of_cells[-1] +"\u25ae" * 50 + " 100 %\n" )
displacements = np.array(displacements)


# write nessesary intermediate information into 4 separate files
name_of_dir_to_save = path_hdf5[:-5]+"/Intermediate files/"

a_file_points = open(name_of_dir_to_save+"itm_Dict_for_number_of_points.pkl", "wb")
pickle.dump(Dict_for_number_of_points, a_file_points)                                    # Save information of number of points in each cluster
a_file_points.close()

a_file_angles = open(name_of_dir_to_save+"itm_Dict_for_rotation_angles.pkl", "wb")
pickle.dump(Dict_for_rotation_angles, a_file_angles)                                     # Save information of angle of rotation for each cluster
a_file_angles.close()

np.save(name_of_dir_to_save+"itm_Displasements", displacements)                          # Save information about displasements in each cluster

np.save(name_of_dir_to_save+"itm_List of Cells", list_of_cells)                          # Save list of clusters for wgich we calculated displasements 

print("\nSucsessfully saved intermediate files")

Using the following probability density function (pdf) based on Rayleigh distribution: 

$$ P(r,a)=\frac{2r}{a}e^{-\frac{r^2}{a}} $$

where $a=4\Delta t D$ and displacements $r$ we can use maximum-likelihood estimation to find diffusion coefficient $D$.

Some displacements originate from noise. The probability density of such displacements should be added to the probability density function (pdf) as a linear summation $br$.

$$  P(r,a)=\frac{2r}{a}e^{-\frac{r^2}{a}}+br $$

When b is a positive, non zero, number the total total probability density will exceed 1.0. Also, when restricting r_max, the total density at r_max may be lower than 1.0. Since we consider displacements limited by $r_{max}$, $P(r,a,b)$ should be normalized by $\int_{0}^{r_{max}} P(r,a,b)\,dx$.

$$ \int_{0}^{r_{max}} \frac{2r}{a}e^{-\frac{r^2}{a}}+br \,dx = \int_{0}^{r_{max}} \frac{2r}{a}e^{-\frac{r^2}{a}} \,dx +\int_{0}^{r_{max}} br \,dx = 1 - e^{-\frac{r_{max}^2}{a}} + \frac{r_{max}^2b}{2}$$


Since the total probability density is 1 the normalized probability density fuction can be written as:

$$ P(r,a,b)=\frac{\frac{2r}{a}e^{-\frac{r^2}{a}}+br}{1 - e^{-\frac{r_{max}^2}{a}}+ \frac{r_{max}^2b}{2}} $$


In [ ]:
###
###              IF RUNNING STARTING ONLY FROM THIS PART OF ANALYSIS, PLEASE REMEMBER TO INITIALIZE PARAMETERS AT THE VERY BEGINING OF A SCRIPT
###              IT WILL DELETE SOME OF THE INFORAMTION IN A ANALUSIS OUTPUT FILDER
###

# loading nessesary data drom previous analysis

name_of_dir_to_load= path_hdf5[:-5]+"/Intermediate files/"

with open(name_of_dir_to_load+'itm_Dict_for_number_of_points.pkl', 'rb') as handle_points:
    Dict_for_number_of_points = pickle.load(handle_points)  

with open(name_of_dir_to_load+'itm_Dict_for_rotation_angles.pkl', 'rb') as handle_angles:
    Dict_for_rotation_angles = pickle.load(handle_angles)

displacements = np.load(name_of_dir_to_load+'itm_Displasements.npy')
list_of_cells = np.load(name_of_dir_to_load+'itm_List of Cells.npy')


import scipy.optimize
import scipy.integrate

def pdf(r, r_max, delta_t, D, b):
    """
    PDF for fitting issues
    """
    a = 4 * delta_t * D
    density = ((2 * r) / a) * np.exp(-(r ** 2) / a) + (b * r)
    normalization = (1 - np.exp(-(r_max ** 2) / a)) + ((r_max ** 2) * b) / 2
    return density / normalization

def likelihood(r, r_max, delta_t, D, b):
    """
    Getting summs of the Logs for MLE 
    """
    return np.sum(np.log(pdf(r, r_max, delta_t, D, b)))
    
def minimization_function(x, *args):
    """
    Minimization of the MLE 
    Returns reversed value of likelihhod
    """
    if background_correction:
        D, b = x
    else:
        D, b = *x, 1
        
    r, r_max, delta_t = args
    return -likelihood(r, r_max, delta_t, D, b)

def maximum_likelihood(r, r_max, delta_t, D, b=0):
    """
    Calculating parameters of the distribution (Diffusion coefficient and background correction coefficient)
    Maximum of the likelihood function is evaluated by minimization of - likelihood function "minimization_function(x, *args)"
    """
    
    if background_correction:
        bounds = [(1e-6, None), (0, None)]
        initial = (D, b)
    else:
        bounds =  [(1e-6, None)]
        initial = (D,)
        
    res = scipy.optimize.minimize(minimization_function, initial, (r, r_max, delta_t), method='Nelder-Mead', bounds=bounds)
    
    return res.x

  
rows = len(list_of_cells)
cols = 2
fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize = (19, rows*7))


# creating a dataframe to save output data as .csv file
df = pd.DataFrame(columns=['Cell_id', 'Number_of_points','Angle_of_rotation', 'Direction_of_rotation', 'Number_of_displasements', "D_value", "b_value", "length, μm", "height, μm"])
name_for_df = path_hdf5[:-5]+", Cells info (sep = %.1f ms, pixel size  = %.0f nm).csv" % (delta_t*1000, int(pixel_size*1000)) 


for i, c in enumerate(list_of_cells):                                                                  
    
    fig_to_save, axes_to_save = plt.subplots(nrows = 1, ncols = 2, figsize = (19, 7))
    
    cell_displacements = displacements[:,1] == c
    _, _, x1, y1, x2, y2, r = displacements[cell_displacements].T
    
    D, b = maximum_likelihood(r, r_max, delta_t, initial_D, initial_b)
    
    x_fitted = np.linspace(0, r_max, 100)
    y_fitted = pdf(x_fitted, r_max, delta_t, D, b)

    if len(list_of_cells) == 1:
        ax = axes[0]
    else:
        ax = axes[i, 0]
    ax.hist(r, density=True, bins="doane", label="Binned displacements", alpha = 0.6)
    axes_to_save[0].hist(r, density=True, bins="doane", label="Binned displacements", alpha = 0.6)
    
    if background_correction:
        label_g = "PDF with b=%.2f" % b
    else:
        label_g = ""

    ax.plot(x_fitted, y_fitted, label=label_g, c = "red", linewidth=2)
    
    y_fitted_one = pdf(np.linspace(0, r_max, 100), r_max, delta_t, D, b)
    ax.plot(np.linspace(0, r_max, 100), y_fitted_one, c="red", label=(r"D=%.2f" % (D,)), linewidth=2)
    axes_to_save[0].plot(np.linspace(0, r_max, 100), y_fitted_one, c="red", label=(r"D=%.2f" % (D,)), linewidth=2)
            
    Row_to_add_to_df = []
    Row_to_add_to_df.append(int(c))
    Row_to_add_to_df.append(Dict_for_number_of_points[c])
    Row_to_add_to_df.append(abs(Dict_for_rotation_angles[c]))
    if Dict_for_rotation_angles[c] >= 0:
        Row_to_add_to_df.append('Clockwise')
    else:
        Row_to_add_to_df.append('Counterclockwise')
    Row_to_add_to_df.append(len(x1))
    Row_to_add_to_df.append(round(D,2))
    Row_to_add_to_df.append(round(b,2))
    Row_to_add_to_df.append(np.amax(x1)-np.amin(x1))
    Row_to_add_to_df.append(np.amax(y1)-np.amin(y1))
    Row_as_serias = pd.Series(Row_to_add_to_df, index = df.columns)
    df = df.append(Row_as_serias, ignore_index=True)
        
    ax.set(xlabel='Displacement', ylabel='Probability', title="Cell %d" % c)
    axes_to_save[0].set(xlabel='Displacement', ylabel='Probability', title="Cell %d" % c)
    ax.grid()
    axes_to_save[0].grid()
    ax.legend(fontsize=15)
    axes_to_save[0].legend(fontsize=15)

    if len(list_of_cells) == 1:
        ax = axes[1]
    else:
        ax = axes[i, 1]
    ax.scatter(x1, y1)
    axes_to_save[1].scatter(x1, y1, c="orange")
    ax.set(xlabel='\u03BCm', ylabel='\u03BCm', title='Localizations of '+ '{0:.0f} start positions of displacements'.format(len(x1)))
    axes_to_save[1].set(xlabel='\u03BCm', ylabel='\u03BCm', title='Localizations of '+ '{0:.0f} start positions of displacements'.format(len(x1)))
    ax.axis('equal')
    axes_to_save[1].axis('equal')
    ax.grid()
    axes_to_save[1].grid()
    name_of_dir_to_save_plots = path_hdf5[:-5]+"/Plots/Global fitting of cells/"
    plt.savefig(name_of_dir_to_save_plots+path_hdf5[:-5]+", Fitting of Cell %a.pdf" % c)
    plt.close(fig_to_save)

dir_to_save_csv = path_hdf5[:-5]+"/CSV files/"
    
df.to_csv(dir_to_save_csv + name_for_df, index = False, header=True)
plt.show()


os.chdir(root_path)

## Make displacement map and analyze cell regions if needed

In [ ]:
class Binning2D:
    """
        This class bins the data by taking all displacements that lie within a given rectangle (x0, y0, x1, y1).
        The class also contains methods to get the minimum and maximum coordinates (get_bounds) and to get a grid of bins (get_bins).
        
        The get_bins method takes an area (x0, y0, x1, y1) and a pixel size (bin_width) and returns a matrix where each column consists of a list of displacements
        (i.e. the upper left column contains a list of displacements that start on a coordinate that falls within (x0, y0, x0 + bin_width, y0 + bin_width)) .
    """
    def __init__(self, x, y, data):
        self.data = data
        self.x = x
        self.y = y
        self.x_indices = np.argsort(self.x)
        self.y_indices = np.argsort(self.y)
        self.x_values_sorted = np.sort(self.x)
        self.y_values_sorted = np.sort(self.y)
    
    def get_data(self, x0, y0, x1, y1):
        x_left = np.searchsorted(self.x_values_sorted, x0, side="left")
        x_right = np.searchsorted(self.x_values_sorted, x1, side="right")
        y_left = np.searchsorted(self.y_values_sorted, y0, side="left")
        y_right = np.searchsorted(self.y_values_sorted, y1, side="right")
        
        indices = np.intersect1d(self.x_indices[x_left:x_right], self.y_indices[y_left:y_right])
        return self.x[indices], self.y[indices], self.data[indices]
    
    def get_bounds(self):
        x_min, x_max = self.x_values_sorted[0], self.x_values_sorted[-1]
        y_min, y_max = self.y_values_sorted[0], self.y_values_sorted[-1]
        return x_min, y_min, x_max, y_max
    
    def get_bins(self, x0, y0, x1, y1, pixel_bin_size):
        rows = []
        for y in np.arange(y0, y1, pixel_bin_size):    
            cols = []
            for x in np.arange(x0, x1, pixel_bin_size):
                cols.append(self.get_data(x, y, x + pixel_bin_size, y + pixel_bin_size))
            rows.append(cols)
        return rows

    
def Cell_slicing():
    """
    Divides the analysid clustered pointcloud in three areas based on the "persantage_of_pole" based on the binning size of the points
    Calculates diffusion coefficients and backgroung correction in each area, plots them and saves in named folders for each cell and area
    """
    
    # initializing the borderes of an analysed cell and borders of areas
    
    width_of_poles = int(round(width*persantage_of_pole,0))
    width_center = int(width - 2 * width_of_poles)

    area_boundarys = [[(0,start_offset),(height, start_offset+width_of_poles)],
                      [(0,start_offset+width_of_poles+1),(height,start_offset+width_of_poles+width_center)],
                      [(0,start_offset+width_of_poles+width_center+1),(height,start_offset+width_of_poles+width_center+width_of_poles)]]
    
    
    x_area_borders = []
    x_area_borders.append(area_boundarys[0][0][1])
    for i in range(0, len(area_boundarys)):
        x_area_borders.append(area_boundarys[i][1][1])
    x_area_borders = [binning.get_bounds()[0] + i * pixel_bin_size for i in x_area_borders]
    
    y_area_borders = []
    y_area_borders.append(area_boundarys[0][0][0])
    y_area_borders.append(area_boundarys[0][1][0])
    y_area_borders = [binning.get_bounds()[1] + i * pixel_bin_size for i in y_area_borders]


    for area_number in range(0,3): 
        
        try:
            os.mkdir("Area %a" % (area_number +1))
        except FileExistsError:
            pass
        dir_to_save = "Area %a" % (area_number+1)
        
        # initializing x,y and r in selected area
        
        x1_area = x1[x1 > x_area_borders[area_number]]
        x1_area = x1_area[ x1_area < x_area_borders[area_number+1]]

        r_area = np.where(x1>x_area_borders[area_number], r, np.nan)
        r_area = np.where(x1<x_area_borders[area_number+1], r_area, np.nan)
        r_area = r_area[np.logical_not(np.isnan(r_area))]

        y_area = np.where(x1>x_area_borders[area_number], y1, np.nan)
        y_area = np.where(x1<x_area_borders[area_number+1], y_area, np.nan)
        y_area = y_area[np.logical_not(np.isnan(y_area))]
    
        # plotting the maps with area borders and scatter plot of analysed displasements
        
        plt.figure(figsize=(15,2))
        plt.subplot(1, 2, 1)
        plt.imshow(fitted_bins, origin="lower", aspect='equal', extent=[binning.get_bounds()[0],
                                                                    binning.get_bounds()[0] + pixel_bin_size * len(fitted_bins[0]),
                                                                    binning.get_bounds()[1],
                                                                    binning.get_bounds()[1] + pixel_bin_size * fitted_bins.shape[0]])
        plt.vlines(x_area_borders, min(y_area_borders),max(y_area_borders), color = "red")
        plt.title('Diffusion map')
        plt.colorbar()
        plt.subplot(1, 2, 2)
        plt.imshow(counts, origin="lower", aspect='equal', extent=[binning.get_bounds()[0],
                                                                        binning.get_bounds()[0] + pixel_bin_size * len(fitted_bins[0]),
                                                                        binning.get_bounds()[1],
                                                                        binning.get_bounds()[1] + pixel_bin_size * fitted_bins.shape[0]])
        plt.vlines(x_area_borders, min(y_area_borders),max(y_area_borders), color = "red")
        plt.title('Number of displacements, area %a' % (area_number+1))
        plt.colorbar()
        plt.scatter(x1_area, y_area, s = 2, c="red")
        plt.savefig(dir_to_save+"/"+path_hdf5[:-5]+", Diffusion map and displasement map + scatter of analysed points, cell %a, area %a" % (c,area_number+1))                     
        plt.show()
        
        # Fitting displacements in selected area
        
        D, b = maximum_likelihood(r_area, r_max, delta_t, initial_D, initial_b)
        x_fitted = np.linspace(0, r_max, 100)
        y_fitted = pdf(x_fitted, r_max, delta_t, D, b)
        
        # plotting the PDF MLE fitting
        
        fig, axes = plt.subplots(figsize=(9,7))
        axes.hist(r_area, density=True, bins="doane", alpha = 0.6)
        axes.plot(np.linspace(0, r_max, 100), y_fitted, label=(r"$D=%.2f\: \frac{\mu m^{2}}{s}, b=%.2f$" % (D, b)), linewidth=2)
        axes.legend(fontsize=14)
        axes.set(xlabel='Displacement', ylabel='Probability', title="Cell %a, Area %a" %(c, area_number+1))
        plt.savefig(dir_to_save+"/"+path_hdf5[:-5]+", Fitting, cell %a, area %a" % (c, area_number+1))
        plt.show()

        
        Row_to_add_to_df_area.append(round(D,2))
        Row_to_add_to_df_area.append(len(x1_area))
        Row_to_add_to_df_area.append(round(b,2))

        
if Area_selection:        
    df_area = pd.DataFrame(columns=['Cell_id', 'D_of_area_1', 'Number_of_points_in_area_1', 'b_of_area_1',
                                               'D_of_area_2', 'Number_of_points_in_area_2', 'b_of_area_2', 
                                               'D_of_area_3', 'Number_of_points_in_area_3', 'b_of_area_2','Side'])
    name_for_df_area = path_hdf5[:-5]+", Areas analysis (bin pixel size = %a nm, poles are %a %%).csv" % (pixel_bin_size_nm, int(persantage_of_pole*100))
    
    os.chdir(path_hdf5[:-5])
    os.chdir('Plots')
    try:
        os.mkdir("Fitting of areas")
    except FileExistsError:
        pass
    os.chdir('Fitting of areas')
    for i in list_of_cells:
        try:
            os.mkdir("Area selection in cell %a" % i)
        except FileExistsError:
            pass
    os.chdir(root_path)
        
for i, c in enumerate(list_of_cells):
    
    cell_displacements = displacements[:,1] == c
    _, _, x1, y1, x2, y2, r = displacements[cell_displacements].T

    x_from = min(np.min(x1), np.min(x2))
    y_from = min(np.min(y1), np.min(y2))
    x_to = max(np.max(x1), np.max(x2))
    y_to = max(np.max(y1), np.max(y2))

    binning = Binning2D(x1, y1, r)
    bins = binning.get_bins(x_from, y_from, x_to, y_to, pixel_bin_size)
    counts = np.array([[len(r) for x, y, r in cols] for cols in bins])
    fitted_bins = []
    fitted_b = []


    for row in bins:
        fitted = []
        for x, y, r in row:
            if len(r) > 10: # make sure we have at least 10 displacements
                D, b = maximum_likelihood(r, r_max, delta_t, initial_D, initial_b)
                fitted.append(D)
                fitted_b.append(b)
            else:
                fitted.append(np.nan)
        fitted_bins.append(fitted)
        
    fitted_bins = np.array(fitted_bins)
    
        
    # Setting the boudarys for Tukey's fences outliers replacement for a median of a dataset
    
    if outlier_correction:
        lower_boundary = np.nanquantile(fitted_bins, 0.25) - k_value * (np.nanquantile(fitted_bins, 0.75) - np.nanquantile(fitted_bins, 0.25))
        upper_boundary = np.nanquantile(fitted_bins, 0.75) + k_value * (np.nanquantile(fitted_bins, 0.75) - np.nanquantile(fitted_bins, 0.25))
        
        fitted_bins = np.where((fitted_bins < lower_boundary) | (fitted_bins > upper_boundary), np.nanmedian(fitted_bins), fitted_bins)                         

    fig = plt.figure(figsize=(15,2))
    
    ax = fig.add_subplot(1, 2, 1)
    ax.set_facecolor('#272C36')                                                                                                                         
    plt.imshow(fitted_bins, origin="lower", aspect='equal', extent=[x_from, x_to, y_from, y_to], cmap = "bwr")
    plt.title('Diffusion map')
    plt.colorbar()
    plt.subplot(1, 2, 2)
    plt.imshow(counts, origin="lower", aspect='equal', extent=[x_from, x_to, y_from, y_to])
    plt.title('Number of displacements')
    plt.colorbar()
    path_to_save = path_hdf5[:-5]+"/Plots/Diffusion maps/"
    plt.savefig(path_to_save+path_hdf5[:-5]+", Diffusion map and displasements map of cell %a.pdf" %c)
    plt.show()
    
    print("Size of a bin = %a nm" % pixel_bin_size_nm)

    summary = ""
        
    # check if all fitted_bins are NaN and crates a terminal output
    if np.all(np.isnan(fitted_bins)):
        summary = "Not enough displacements per bin. All bins are NaN."
    else:
        summary  = "Mean diffusion coefficient               : %.2f um^2/s\n" % np.nanmean(fitted_bins)
        summary += "Standard deviation diffusion coefficient : %.2f um^2/s\n" % np.nanstd(fitted_bins)
        summary += "Minimum diffusion coefficient            : %.2f um^2/s\n" % np.nanmin(fitted_bins)
        summary += "Maximum diffusion coefficient            : %.2f um^2/s\n" % np.nanmax(fitted_bins)
        summary += "Mean number of displacements per bin     : %d\n" % counts.mean()
        summary += "Mean background                          : %.2f\n" % np.mean(fitted_b)
        summary += "Standard deviation background            : %.2f\n" % np.std(fitted_b)
    print("Cell number %a" % (c))
    if outlier_correction:
        print("K-value is %a" % k_value)
    print(summary)
    
    if Area_selection:          
        os.chdir(path_hdf5[:-5])
        os.chdir('Plots')
        os.chdir('Fitting of areas')
        os.chdir('Area selection in cell %a' % c)

        Row_to_add_to_df_area = []
        Row_to_add_to_df_area.append(int(c))

        cell_displacements = displacements[:,1] == c
        _, _, x1, y1, x2, y2, r = displacements[cell_displacements].T
        width = fitted_bins.shape[1]
        height = fitted_bins.shape[0]
        start_offset = 0
        Cell_slicing()

        os.chdir(root_path)

        Row_to_add_to_df_area.append("As one cell")
        Row_as_serias = pd.Series(Row_to_add_to_df_area, index = df_area.columns)
        df_area = df_area.append(Row_as_serias, ignore_index=True)
                            
# write areas data in a .csv file
if Area_selection:
    dir_to_save_csv = path_hdf5[:-5]+"/CSV files/"
    df_area.to_csv(dir_to_save_csv+name_for_df_area, index = False, header=True)